# 2.  Integration Techniques
Often physical scientists desire the integration of a function. An analytical integration may not be achievable and a numerical solution is required. Here, we will explore the midpoint, trapezoidal, Simpson's, Gaussian quadrature, and Monte Carlo numerical integration methods.

In the next cell, the necessary packages are imported, such as NumPy needed to perform certain mathematical operations, and Matplotlib for visualization. The Sympy library helps define the mathematical functions we would like to explore.  For example, when defining functions, you may want to explore functions not included in Python by default, such as $sin()$, $cos()$, but these functions are available through Sympy which we can access by typing `syp.sin()` and `syp.cos()`.
We also define the specific function that we want with each integration method.
By default the function is defined as $sin(x)$.

## Your Task!
1. Change the defined function from sine to cosine while using the Sympy library.


In [ ]:
###########
# IMPORTS #
###########
# These are packages needed to perform the math and visualization

import numpy as np
import sympy as syp
import IPython
from IPython.display import Math
import matplotlib.pyplot as plt
import plotting_functions as pf
%matplotlib notebook

#############
# FUNCTIONS #
#############


def f(x):
    '''
    INPUT:
        x (float) array of floats
    OUTPUT: 
        returns (float)
    '''
    return syp.sin(x)



###############
# GLOBAL VARS #
###############
a = 0  # left most point of interval
b = np.pi  # right most point of interval
n = 100 # number of subsections
# This will calculate the analytical integral to assess error
analytical_integral = pf.analytical_integral(f,a,b)

# dictionaries to hold error of each method
error = dict()

## ** 2.1  Midpoint Rule **

The simplest numerical method for approximating integrals is the midpoint rule.  This method approximates $f(x)$ with a polynomial $p(x)$ over a particular interval.

$$ \int_a^b f(x)dx \ \approx \ \int_a^b p(x)dx $$

This polynomial is just the value of the function at a single point, the midpoint $mp$ between $a$ and $b$.

$$ mp\ =\ \frac{a+b}{2} $$

$$ \int_a^b p(x)dx \ = \ \int_a^b f(mp)dx \ = \ (b-a) f(mp) $$ 

For the midpoint method to yield a reasonable result, the interval must be broken into several subintervals, apply the midpoint rule to each subinterval, and sum over all subintervals. This implementation is known as the composite midpoint rule.

In the cell below, we define the `midpoint` function that assess the function f at the midpoint between $a$ and $b$. A second function `Cmidpoint`, implements the composite midpoint rule. 

In [ ]:
#############
# FUNCTIONS #
#############


def midpoint(a, b, f):  
    '''
    Midpoint rule evaluation of the function, called in Cmidpoint below
    INPUT
        a: (float) minimum x-value to evaluate
        b: (float) maximum x-value to evaluate
        f: function
    OUTPUT
        returns (float) integral of f evaluated as the midpoint of a and b
    '''
    #calculating the midpoint between the interval a and b
    mp = (a + b) / 2.0
    #approximating the value of the integral for that subsection with the midpoint method
    return (b - a) * f(mp)


def Cmidpoint(a, b, f, n_sub):
    '''
    Composite midpoint calculation
    INPUT
        a: (float) minimum x-value to evaluate
        b: (float) maximum x-value to evaluate
        f: function
        n_sub: (int) number of subsections between [a,b] to evaluate
    OUTPUT
        returns (float) integral of f evaluated as the sum of each subsections midpoint
    '''
    # dividing the domain of the function (a to b) into subsections (Nsub)
    xvals = np.linspace(a, b, n_sub + 1)
    p = 0
    for i in range(n_sub):
        # approximating the integral by summing the midpoint evaluation for
        # each subsection
        p += midpoint(xvals[i], xvals[i + 1], f)
    return p

#############
# MAIN CODE #
#############
integral_estimator = Cmidpoint(a, b, f, n)
method = 'Midpoint Approximation'
error[method] = abs(integral_estimator - analytical_integral)
print('Method: {}'.format(method))
print('Answer = {:.6E}'.format(integral_estimator))
print('Number of evaluated points = {}'.format(n))


A geometrical interpretation of this method can help us learn more about these methods. Below, we will plot the results of the integration as assessed with the midpoint rule. The dotted line is the midpoint of the subsection, from which the rectangular subsection will approximate the integral. Notice how the rectangles will sometimes over and under estimate the function. 

## Your task
1. Run the cell below that will use the composite midpoint rule. It will break the section a and b (defined above) into n_sub subsection, initially set to 8.
2. Change the variable n_sub from 8 to 25 and run this again.
3. Change the variable n_sub from 25 to 100 and observe how the plot change.

In [ ]:
# Input variables are:
n_sub = 8
# (x minimum, x maximum, function, number of subsections)
pf.plot_midpoint_func(a, b, f, n_sub)
integral_estimator = Cmidpoint(a, b, f, n_sub)
print('Answer = {:.6E}'.format(integral_estimator))
print('Error = {:.6E}'.format(abs(integral_estimator - analytical_integral)))

## 2.2  Trapezoidal Rule 

As an improvement to the midpoint rule, the trapezoidal rule can also be utilized to approximate integrals.  This method utilizes two points instead of the single point approximation in the midpoint method. A linear interpolation polynomial, $p(x)$, is used to approximate our desired function, $f(x)$.  The two points utilized in the approximation will be the endpoints of the interval: $(a, f(a))$ and $(b, f(b))$.

$$ p(x) = f(a)\frac{x\ -\ b}{a\ -\ b}\ +\ f(b)\frac{x\ -\ a}{b\ -\ a} $$

The approximation of our desired function then becomes:
$$ \int_a^b f(x)dx\ \approx\  \int_a^b p(x)dx\ = \left[f(a)\frac{x\ -\ b}{a\ -\ b}\ +\ f(b)\frac{x\ -\ a}{b\ -\ a}\right]dx $$

which can be re-written as:

$$ \int_a^b f(x)dx\ \approx\  \int_a^b p(x)dx\ = \frac{f(a)}{-h}\int_a^b(x\ -\ b)dx\ +\ \frac{f(b)}{h}\int_a^b(x\ -\ a)dx $$

where $$ h\ =\ b\ -\ a $$

and evaluating the integrals gives rise to

$$ \int_a^b f(x)dx\ \approx\ \frac{f(a)}{-h}\frac{(x\ -\ b)^2}{2}\Bigm|_a^b\ +\ \frac{f(b)}{h}\frac{(x\ -\ a)^2}{2}\Bigm|_a^b $$

Simplifying this expression results in:

$$ \int_a^b f(x)dx\ \approx\ \frac{hf(a)}{2}\ +\ \frac{hf(b)}{2} = \frac{h}{2}\left[f(a)\ +\ f(b)\right] $$

A composite trapezoidal rule breaks up our interval into several subintervals, applies the trapezoidal rule to each subinterval, and sums over all subintervals, in much the same way as the composite midpoint rule. The cell defines a function that will evaluate an approximation to the integral $f(x)$ using  the composite trapezoid rule.

In [ ]:
#############
# FUNCTIONS #
#############

def trapezoid(a, b, f):
    '''
    Calculates the trapezoid rule on a single section
    INPUT
        a: (float) minimum x-value to evaluate
        b: (float) maximum x-value to evaluate
        f: function
    OUTPUT
        ans: (float) integral of f evaluated as area of the trapezoid between a and b
    '''
    h = (b - a)
    estimate = (f(a)+f(b))/2  # the final evaluation of f(x)
    ans = estimate * h
    return ans

def Ctrapezoid(a, b, f, n_sub):
    '''
    Calculates the composite trapezoid rule over the subsection a and b split into multiple subsections
    INPUT
        a: (float) minimum x-value to evaluate
        b: (float) maximum x-value to evaluate
        f: function
        n_sub: (int) number of subections between [a,b] to evaluate
    OUTPUT
        returns (float) integral of f evaluated as the sum of trapezoidal area of each subsection
    '''
    xvals = np.linspace(a, b, n_sub + 1)
    estimate = 0
    for i in range(n_sub):
        estimate += trapezoid(xvals[i],xvals[i+1],f)
    return estimate


#############
# MAIN CODE #
#############
integral_estimator = Ctrapezoid( a, b, f, n)
method = 'Trapezoidal Rule'
error[method] = abs(integral_estimator - analytical_integral)
print("Method: {}".format(method))
print('Answer = {:.6E} +/- {:.6E}'.format(integral_estimator, error[method]))

Now that we have seen how to code the trapezoid rule, what does it actually look like in practice? The plot below will show a geometric interpretation of the trapezoid rule. Here the black line represents our function. Each vertical red line are the end points, $a$ and $b$, that form the sides of the trapezoid. Notice how some parts of the curve are over-represented as a result of the trapezoid being taller than the function. However, there are also areas in which the trapezoid falls beneath the function, resulting in a lower area than required to describe the function. The success of the trapezoidal rule is reliant on the cancellation of these over and under estimations. 

## Your task
1. Run the cell below that will use the composite midpoint rule. It will break the section a and b (defined above) into n_sub subsection, initially set to 8.
2. Change the variable n_sub from 8 to 25 and run this again.
3. Change the variable n_sub from 25 to 100 and observe how the plot changes.

In [ ]:
#input variables are:
n_sub = 8
#(x minimum, x maximum, function, number of subsections)
pf.plot_trapezoidal_func(a, b, f, n_sub)
integral_estimator = Ctrapezoid( a, b, f, n_sub)
print('Answer = {:.6E}'.format(integral_estimator))
print('Error = {:.6E}'.format(abs(integral_estimator - analytical_integral)))

##  2.3  Simpson's Rule 

For an attempt at a more accurate approximation, a three-point approximation can be utilized, where a quadratic interpolation polynomial, $p(x)$ represents our function, $f(x)$.  In this case, our three points are $(a, f(a))$, $(b, f(b))$, and $(mp, f(mp))$, where $mp$ is the midpoint between $a$ and $b$. 

The quadratic interpolation polynomial takes on the form:

$$ p(x)\ =\ f(a)\frac{(x-mp)(x\ -\ b)}{(a\ -\ mp)(a\ -\ b)}\ +\ f(mp)\frac{(x\ -\ a)(x\ -\ b)}{(mp\ -\ a)(mp\ -\ b)}\ +\ f(b)\frac{(x\ -\ a)(x\ -\ mp)}{(b\ -\ a)(b\ -\ mp)} $$

where 

$$ \int_a^b p(x)dx\ =\ f(a)\int_a^b\frac{(x\ -\ mp)(x\ -\ b)}{(a\ -\ mp)(a\ -\ b)}dx\ +\ f(mp)\int_a^b\frac{(x\ -\ a)(x\ -\ b)}{(mp\ -\ a)(mp\ -\ b)}dx\ $$
$$ +\ f(b)\int_a^b\frac{(x\ -\ a)(x\ -\ mp)}{(b\ -\ a)(b\ -\ mp)}dx $$


which can be simplified to:

$$ \int_a^b f(x)dx\ \approx\ \frac{h}{3}\left[\ f(a)\ +\ 4f(mp)\ +\ f(b)\ \right] $$

where

$$ mp\ =\ \frac{a\ +\ b}{2} $$ and $$ h\ =\ \frac{b\ -\ a}{2} $$

In a similar fashion to the midpoint and trapezoidal rules, a composite Simpson's rule can also be applied, where the interval is broken into several subintervals, the Simpson's rule is applied to each subinterval, and the sum over all subintervals is taken as the approximate integration.

In [ ]:
#############
# FUNCTIONS #
#############


def Simpson(a, b, f):  #approximates the function between a and b as a polynomial
    '''
    INPUT
        a: (float) minimum x-value to evaluate
        b: (float) maximum x-value to evaluate
        f: function
    OUTPUT
        returns (float)
    '''
    m = (a + b) / 2.0
    h = abs(b - a) / 2.0
    return h / 3.0 * (f(a) + 4.0 * f(m) + f(b))


def Csimpson(a, b, f, n_sub):
    '''
    INPUT
        a: (float) minimum x-value to evaluate
        b: (float) maximum x-value to evaluate
        f: function
        n_sub: (int) number of subsections between [a,b] to evaluate
    OUTPUT
        returns (float)
    '''
    xvals = np.linspace(a, b, n_sub + 1)
    estimate = 0
    for i in range(n_sub):
        # summing the approximations of f(x) for each subsection
        estimate = estimate + Simpson(xvals[i], xvals[i + 1], f)
    return estimate
#############
# MAIN CODE #
#############
integral_estimator = Csimpson(a, b, f, n)
method = 'Simpson\'s Rule'
error[method] = abs(integral_estimator - analytical_integral)
print("Method: {}".format(method))
print('Answer = {:.6E} +/- {:.6E}'.format(integral_estimator, error[method]))


The function in the following code block serves to give a geometric interpretation of the Simpson's rule approximation. Here the red points represent $(a,f(a))$ and $(b,f(b))$. The larger blue points represent the function evaluated at the midpoint of $a$ and $b$. A dotted line between the three points represents the estimated polynomial for that interval of the function. 

In [ ]:
#input variables are:
#(x minimum, x maximum, function, number of parabola)
pf.plot_simpsons_func(a, b, f, 4)

## 2.4  Gaussian Quadrature 
The previous approximations dealt with estimating integrals as simple polynomials evaluated at points uniformly distributed within the interval [a,b]. Gaussian quadrature follows the concept by selecting *optimally* distributed points according to: 
$$ \int_a^b f(x)dx\ \approx\ \sum_{i\ =\ 1}^{n} w_i f(x_i) $$
where $w_i$ are the weighting factors. The Gaussian quadrature weights may be found using Legendre polynomials ($P_n$), belonging to the family of *orthogonal polynomials*. The Legendre polynomials have roots at $x_i$, which become the optimal sampling points. Although Gaussian quadrature estimation is defined over the interval [-1,1], it becomes universal through a change of variables:
$$ \int_a^b f(x)dx\ \approx\ \frac{b\ -\ a}{2}\sum_{i\ =\ 1}^{n} w_i f\left(\frac{b\ -\ a}{2} x_i\ +\ \frac{a\ +\ b}{2}\right) $$
where
$$ w_i\ =\ \frac{2}{(1\ -\ x_i^2)[P_n'(x_i)]^2} $$
The optimally chosen points, $x_i$, are now the $i^{th}$ root of $P_n$.

In [ ]:
#############
# FUNCTIONS #
#############

def gaussian_quadrature(a, b, f, n):
    '''
    INPUT
        a: (float) minimum x-value to evaluate
        b: (float) maximum x-value to evaluate
        f: function
        n: (int) number of points to evaluate
    OUTPUT
        returns (float)
    '''
    # determination of optimal points and weights for evaluation as 
    # the roots of the Legendre polynomials
    x, w = np.polynomial.legendre.leggauss(n)
    # change of variables for [-1,1] to [a,b]
    x = ((b - a) / 2.0) * x + ((b + a) / (2.0))
    try: 
        estimate = ((b - a) / 2.0) * np.sum(w * f(x))
    except:
        f_temp = pf.sympy2numpy_function_converter(f,x)
        estimate = ((b - a) / 2.0) * np.sum(w * f_temp(x))
    return estimate

#############
# MAIN CODE #
#############
integral_estimator = gaussian_quadrature(a, b, f, 8)
method = 'Gaussian Quadrature'
error[method] = abs(integral_estimator - analytical_integral)
print("Method: {}".format(method))
print('Answer = {:.6E} +/- {:.6E}'.format(integral_estimator,error[method]))


In the plot below, the roots of the Legendre polynomials, $x_i$, are evaluated for the function $f(x)$ and represented by the red circles. The difference in diameter of each circle represents the relative weight assigned to each point.

In [ ]:
#input variables are:
#(x minimum, x maximum, function, number of points)
pf.plot_gaussian_func(a, b, f, 8)

## 2.5  Monte Carlo 
Monte Carlo methods rely on the law of large numbers to approximate a definite integral; given a large enough sample size, a definite integral $f(x)$ on $[a,b]$ can be approximated by a sum:
$$ \int_a^b f(x)dx\ \approx\ \frac{1}{N} (b-a)\sum_{i\ =\ 1}^{N} f(x_i) $$
Unlike the previous methods that used equally spaced sampling points, Monte Carlo methods draw random numbers in the range of the function to evaluate. The true answer will lie within error bars or the uncertainty measurement of the estimation. The uncertainty will decrease as the number of points sampled increases. The uncertainty is measured as the standard deviation of our results.
The distribution of the sampling points can be drawn in ways to enhance the efficiency of this method. Here we implement random sampling that draws points from a uniform distribution. Improved sampling draws points from different distributions, which could reduce the number of points required to achieve the same numerical precision. 



In [ ]:
#############
# FUNCTIONS #
#############


def MonteCarlo(a, b, f, n):
    '''
    INPUT
        a: (float) minimum x-value to evaluate
        b: (float) maximum x-value to evaluate
        f: function
        n: (int) number of points to evaluate
    OUTPUT
        estimate: (float) of the approximation of the integral of f
        uncertainty: (float) of the uncertainty in the approximation of the integral of f
    '''
    f= pf.sympy2numpy_function_converter(f,n)
    random_samples = np.random.uniform(a, b, n)
    function_values = f(random_samples)
    estimate = (np.sum(function_values) / n) * (b - a)
    uncertainty = np.abs(
        b - a) * np.sqrt(((np.sum(function_values**2) / n) - (np.sum(function_values / n)**2)) / n)
    return estimate, uncertainty
#############
# MAIN CODE #
#############
method = 'Monte Carlo Approximation'
integral_estimator,error[method]= MonteCarlo(a, b, f, n)

print("Method: {}".format(method))
print('Answer = {:.6E} +/- {:.6E}'.format(integral_estimator,error[method]))


The plot below shows the function evaluated with Monte Carlo methods. Unlike the previous methods, the dots are not equally spaced, which reflects the random choices of $x$ used to evaluate 
$f(x)$. The variation in dot color is simply to make them more distinguishable.  

In [ ]:
#input variables are:
#(x minimum, x maximum, function, number of points)
pf.plot_MonteCarlo_func(a, b, f, 8)

## 2.6 Method Summary

A table will appear when you run the code block below that presents the error of each method for the function defined in the first code block.
Of the three approximations with equally spaced abscissa, Simpson's method gives the most accurate approximation, followed by trapezoidal, and midpoint. However, as observed in the test case, the cancellation of errors artificially suggests the midpoint rule performs well. Gaussian quadrature tends to be the most accurate method for 1-dimensional functions. However, the necessity of Legendre polynomials can become costly for certain functions, which may render another method more effective. Monte Carlo methods, which seems to underperform compared to the other methods, is very effective when applied to higher dimensional functions.

In [ ]:
print("{:-^39}".format(""))
print("{:^25} : {:^11}".format("Method", "Error"))
print("{:->25}   {:->11}".format("", ""))
for i in sorted(error, key=error.get, reverse=True):
    print("{:>25} : {:>.5E}".format(i, error[i]))
print("{:-^39}".format(""))


## 2.7  Application



### Particle in a box wavefunctions

Consider the quantum mechanical problem of a particle in a box. This model problem is a single particle with only kinetic energy bound to a box of length < L by an infinitely large potential at and beyond the walls. This system does not have continuous energy levels, but instead only a discrete set that depend on the size and shape of the box.

The functions that can accurately describe this, must oscillate between the walls, but equal zero at the walls. The sine function with a wave number restricted to $\frac{n\pi}{L}$ satisfies this condition. The normalized wavefunction for the particle in a box is where the quantum number $n$ is an integer. Wavefunctions for different quantum numbers must be orthogonal. 
$$\phi_a = \sqrt{\frac{2}{L}}sin(\frac{n_a\pi x}{L}).$$

The following overlap integral ensures the orthonormal constraint is satisfied.
$$\int\ \phi_a \phi_b\ dx= 
\begin{cases} 
0\  \text{if}\ a\ \neq\ b\\
1\  \text{if}\ a\  =\ b
\end{cases}$$


Here, we can use the numerical methods to ensure these functions are orthonormal. We will estimate the overlap of wavefunctions using Gaussian quadrature integration since this method can give reliable results with a modest amount of points compared to the other methods discussed in this notebook. However, in addition to ensuring orthonormality, we will also explore how numerical integration can introduce errors into our calculation if we are not careful. To explore this, we will look at adapting the number of quadrature points used for integration.

### Your task
1. Run the code block below to produce an interactive plot.
2. Scroll down to the following code block to continue this exercise.

In [ ]:
import ipywidgets as ipw
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button

# particle in a box
def particle_in_a_box(x, level, L):
    norm = np.sqrt(2.0/L)
    psi = norm*np.sin(level*np.pi*x/L)
    return psi

# integration
def pib_gaussian_quadrature(a, b, f, n, level1, level2, L):
    '''
    INPUT
        a: (float) minimum x-value to evaluate
        b: (float) maximum x-value to evaluate
        f: function
        n: (int) number of points to evaluate
        level1: (int) level of first wavefunction
        level2: (int) level of second wavefunction
    OUTPUT
        returns (float)
    '''
    # determination of optimal points and weights for evaluation based on
    # legendre polynomials
    x, w = np.polynomial.legendre.leggauss(n)
    # change of variables for [-1,1] to [a,b]
    x = ((b - a) / 2.0) * x + ((b + a) / (2.0))
    try:
        estimate = ((b - a) / 2.0) * np.sum(w * f(x,level1,L) * f(x,level2,L))
    except:
        f_temp = pf.sympy2numpy_function_converter(f,x)
        estimate = ((b - a) / 2.0) * np.sum(w * f_temp(x,level1,L) * f_temp(x,level2,L))
    return estimate

ipw.interact(pf.plyt,level1=(1,10),level2=(1,10),L=ipw.fixed(1),num_pts=(1,30))


### Your Task Continued
Consider the case of $a$ = $b$ = 1
1. What would you expect the overlap of $\phi_a$ and $\phi_b$?
2. What is the result of the overlap for $\phi_a$  and $\phi_b$ if the number of points = 9?
3. What is the result of the overlap for $\phi_a$  and $\phi_b$ if the number of points = 3?
4. How can you explain these results?

Consider the case of  $a$ = 9 and $b$ = 1
1. What would you expect the overlap of $\phi_a$ and $\phi_b$?
2. What is the result of the overlap for $\phi_a$  and $\phi_b$ if the number of points = 9?
3. What is the result of the overlap for $\phi_a$  and $\phi_b$ if the number of points = 3?
4. How can you explain these results?


## 2.8  Extra Examples
Below we apply the described methods to relevant physical chemistry problems taken from McQuarrie and Simon Physical Chemistry: A Molecular Approach textbook. 


### a.

The heat capacity of a monoatomic crystal is given by
$$C_v\ =\ 9R\left(\frac{T}{\Theta_D}\right)^3\int_0^{\Theta_D/T}\frac{x^4e^x}{(e^x-1)^2}dx$$

where $\Theta_D$ is the Debeye temperature, a parameter characteristic of a crystalline substance, and $R$ is the molar gas constant. Given that $\Theta_D\ =\ 309\ \text{K}$ for copper calculate the molar heat capacity of copper at $103\ \text{K}$.

In [ ]:
def f(x):
    cv = 9.0 * 8.314 * (103. / 309.)**3 * ((x**4 * syp.exp(x)) / (syp.exp(x) - 1)**2)
    return cv

a = 1e-6
b = 309 / 103.
n = 100
answer_a = dict()


** Midpoint **

In [ ]:
integral_estimator = Cmidpoint(a, b, f, n)
method = 'Midpoint'
print('Method: {}'.format(method))
print('Answer = {:.6E}'.format(integral_estimator))
answer_a[method] = integral_estimator


In [ ]:
pf.plot_midpoint_func(a, b, f, 8)

** Trapezoidal **

In [ ]:
integral_estimator = Ctrapezoid(a, b, f, n)
method = 'Trapezoidal'
print('Method: {}'.format(method))
print('Answer = {:.6E}'.format(integral_estimator))
answer_a[method] = integral_estimator


In [ ]:
pf.plot_trapezoidal_func(a, b, f, 8)

** Simpson's**

In [ ]:
integral_estimator = Csimpson(a, b, f, n)
method = "Simpson's"
print('Method: {}'.format(method))
print('Answer = {:.6E}'.format(integral_estimator))
answer_a[method] = integral_estimator

In [ ]:
pf.plot_simpsons_func(a, b, f, 8)

** Gaussian Quadrature **

In [ ]:
m = 8
integral_estimator = gaussian_quadrature(a, b, f, m)
method = 'Gaussian'
print('Method: {}'.format(method))
print('Answer = {:.6E}'.format(integral_estimator))
answer_a[method] = integral_estimator

In [ ]:
pf.plot_gaussian_func(a, b, f, 8)

** Monte Carlo **

In [ ]:
integral_estimator, uncertainty = MonteCarlo(a, b, f, n)
method = 'Monte Carlo'
print('Method: {}'.format(method))
print('Answer = {:.6E}'.format(integral_estimator))
print('uncertainty: +/- {:.6E}'.format(uncertainty))
answer_a[method] = integral_estimator


In [ ]:
pf.plot_MonteCarlo_func(a, b, f, 8)

In [ ]:
print("{:-^39}".format(""))
print("{:^25} : {:^11}".format("Method", "Answer"))
print("{:->25}   {:->11}".format("", ""))
for i in sorted(answer_a, key=answer_a.get, reverse=True):
    print("{:>25} : {:>.5E}".format(i, answer_a[i]))
print("{:-^39}".format(""))

### **b.**
 The constant-pressure molar heat capacity of O$_2$(g) from $300 \text{K}$ to $1200 \text{K}$ is given by
 
$$\bar{C_p}(T)/\text{J} \text{K}^{-1}\text{mol}^{-1} = 25.72\ +\ (12.98 \times 10^{-3} \text{K}^{-1})T - (38.62 \times 10^{-7}\text{K}^{-2})T^2$$ 

where $T$ is in kelvin. Calculate the value of $\Delta \bar{S}$, the total entropy change, when one mole of O$_2$ (g) is heated at constant pressure from $300 \text{K}$ to $1200 \text{K}$. 

For this example,$\Delta \bar{S}$ is given by the integral: 
$$\Delta \bar{S} = \int_{T_1}^{T_2} \frac{\bar{C}_P(T)}{T}dT$$
plugging in our expression for $\bar{C_p}(T)$,
$$\Delta \bar{S} = \int\limits_{300\ K}^{1200\ K} \frac{25.72}{T}dT + \int\limits_{300\ K}^{1200\ K} (12.98 \times 10^{-3} \text{K}^{-1})dT - \int\limits_{300\ K}^{1200\ K} (38.62 \times 10^{-7}\text{K}^{-2})TdT$$

In [ ]:
def f(x):
    return (25.72 / x) + 12.98e-3 - 38.62e-7 * x
a = 300
b = 1200
n = 1000
answer_b = dict()

** Midpoint **

In [ ]:
integral_estimator = Cmidpoint(a, b, f, n)
method = 'Midpoint'
print('Method: {}'.format(method))
print('Answer = {:.6E}'.format(integral_estimator))
answer_b[method] = integral_estimator

In [ ]:
pf.plot_midpoint_func(a, b, f, 8)

** Trapezoidal **

In [ ]:
integral_estimator = Ctrapezoid(a, b, f, n)
method = 'Trapezoidal'
print('Method: {}'.format(method))
print('Answer = {:.6E}'.format(integral_estimator))
answer_b[method] = integral_estimator

In [ ]:
pf.plot_trapezoidal_func(a, b, f, 8)

** Simpson's **

In [ ]:
integral_estimator = Csimpson(a, b, f, n)
method = "Simpson's"
print('Method: {}'.format(method))
print('Answer = {:.6E}'.format(integral_estimator))
answer_b[method] = integral_estimator

In [ ]:
pf.plot_simpsons_func(a, b, f, 8)

** Gaussian Quadrature **

In [ ]:
m = 8
integral_estimator = gaussian_quadrature(a, b, f, m)
method = 'Gaussian'
print('Method: {}'.format(method))
print('Answer = {:.6E}'.format(integral_estimator))
answer_b[method] = integral_estimator

In [ ]:
pf.plot_gaussian_func(a, b, f, 8)

** Monte Carlo **

In [ ]:
#############
# MAIN CODE #
#############
integral_estimator, uncertainty = MonteCarlo(a, b, f, n)
method = 'Monte Carlo'
print('Method: {}'.format(method))
print('Answer = {:.6E}'.format(integral_estimator))
print('uncertainty: +/- {:.6E}'.format(uncertainty))
answer_b[method] = integral_estimator

In [ ]:
pf.plot_MonteCarlo_func(a, b, f, 8)

In [ ]:
print("{:-^39}".format(""))
print("{:^25} : {:^11}".format("Method", "Answer"))
print("{:->25}   {:->11}".format("", ""))
for i in sorted(answer_b, key=answer_a.get, reverse=True):
    print("{:>25} : {:>.5E}".format(i, answer_a[i]))
print("{:-^39}".format(""))
